In [32]:
import ccxt
import pandas as pd
from datetime import timedelta
# Define the trading pair symbol
def get_current_dataframe_and_last_5_volume_sum():
    symbol = 'ETH/USDT'

    # Create an instance of the Binance exchange
    exchange = ccxt.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    # Replace 'YOUR_START_TIMESTAMP' and 'YOUR_END_TIMESTAMP' with the desired timestamps
    current_time_utc = datetime.utcnow() - timedelta(hours=1)

    # Format the current time as a string in the required format ('%Y-%m-%dT%H:%M:%SZ')
    start_time_str = current_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Parse the formatted string to create the start_timestamp
    start_timestamp = exchange.parse8601(start_time_str)
    # Fetch historical OHLCV data for the specified trading pair and timeframe
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1m', since=start_timestamp, limit=None)

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df['Timestamp'] = df['Timestamp'] + timedelta(hours=8)
    df['last_5_volume_sum'] = df['Volume'].rolling(window=6, min_periods=1).sum() - df['Volume']
    last_5_volume_sum = df['last_5_volume_sum'].iloc[-1]
    # Print the DataFrame
    return df, last_5_volume_sum

In [33]:
import ccxt.pro as ccxtpro
import ccxt
import asyncio
import nest_asyncio
import pytz
from datetime import datetime
from binance_api import api_key, api_secret
import time

# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

df = get_current_dataframe_and_last_5_volume_sum()[0]

async def fetch_and_print_ohlcv(exchange, symbol):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')

    try:
        previous_timestamp = None  # Variable to store the previous timestamp

        while True:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')

            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)
            timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
            current_timestamp_str = timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')

            # Check if the timestamp has changed
            if current_timestamp_str != previous_timestamp:
                print(f"New timestamp: {current_timestamp_str}")
                df, last_5_volume_sum = get_current_dataframe_and_last_5_volume_sum()

                previous_timestamp = current_timestamp_str  # Update the previous timestamp
            current_volume  = ohlcv[-1][5]
            if current_volume > last_5_volume_sum:
                print(f"last_5_volume_sum: {last_5_volume_sum}, current_volume: {current_volume}")


            print(f"time: {current_timestamp_str}, open: {ohlcv[-1][1]}, high: {ohlcv[-1][2]}, low: {ohlcv[-1][3]}, close: {ohlcv[-1][4]}, volume: {ohlcv[-1][5]}")

            await asyncio.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def main():
    symbol = 'ETH/USDT'
    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    await fetch_and_print_ohlcv(exchange, symbol)

# Run the async/await code in a Jupyter Notebook cell
asyncio.run(main())


New timestamp: 2023-09-28 23:55:00
time: 2023-09-28 23:55:00, open: 1648.6, high: 1651.06, low: 1648.26, close: 1650.8, volume: 4274.898
time: 2023-09-28 23:55:00, open: 1648.6, high: 1651.06, low: 1648.26, close: 1650.57, volume: 4357.183
time: 2023-09-28 23:55:00, open: 1648.6, high: 1651.06, low: 1648.26, close: 1650.76, volume: 4444.875
time: 2023-09-28 23:55:00, open: 1648.6, high: 1651.06, low: 1648.26, close: 1650.8, volume: 4453.786
New timestamp: 2023-09-28 23:56:00
time: 2023-09-28 23:56:00, open: 1650.8, high: 1650.87, low: 1650.8, close: 1650.86, volume: 18.778
time: 2023-09-28 23:56:00, open: 1650.8, high: 1651.33, low: 1650.65, close: 1650.65, volume: 225.201
time: 2023-09-28 23:56:00, open: 1650.8, high: 1651.33, low: 1650.63, close: 1650.66, volume: 283.931
time: 2023-09-28 23:56:00, open: 1650.8, high: 1651.33, low: 1650.63, close: 1650.79, volume: 312.57
time: 2023-09-28 23:56:00, open: 1650.8, high: 1651.33, low: 1650.63, close: 1650.94, volume: 364.372
time: 2023-09

In [27]:
import ccxt.pro as ccxtpro
import ccxt
import asyncio
import nest_asyncio
import pytz
from datetime import datetime
from binance_api import api_key, api_secret
import time
# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

async def fetch_and_print_ohlcv(exchange, symbol):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')

    try:
        while True:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')

            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)
            timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
            # print(ohlcv)
            print(f"time: {timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')}, open: {ohlcv[-1][1]}, high: {ohlcv[-1][2]}, low: {ohlcv[-1][3]}, close: {ohlcv[-1][4]}, volume: {ohlcv[-1][5]}")

            await asyncio.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def main():
    symbol = 'ETH/USDT'
    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    await fetch_and_print_ohlcv(exchange, symbol)

# Run the async/await code in a Jupyter Notebook cell
asyncio.run(main())


time: 2023-09-28 23:15:00, open: 1641.89, high: 1643.15, low: 1641.0, close: 1643.15, volume: 1753.909
time: 2023-09-28 23:16:00, open: 1643.16, high: 1643.16, low: 1643.08, close: 1643.08, volume: 46.383
time: 2023-09-28 23:16:00, open: 1643.16, high: 1643.16, low: 1642.66, close: 1642.66, volume: 156.701
time: 2023-09-28 23:16:00, open: 1643.16, high: 1643.16, low: 1642.66, close: 1642.86, volume: 175.419
time: 2023-09-28 23:16:00, open: 1643.16, high: 1643.16, low: 1642.66, close: 1643.0, volume: 182.756


KeyboardInterrupt: 

In [25]:
import ccxt
import pandas as pd
from datetime import timedelta
# Define the trading pair symbol
symbol = 'ETH/USDT'

# Create an instance of the Binance exchange
exchange = ccxt.binance({
    'options': {
        'defaultType': 'future',  # Set the default market type to 'future'
        'newUpdates': True 
    }
})

# Replace 'YOUR_START_TIMESTAMP' and 'YOUR_END_TIMESTAMP' with the desired timestamps
current_time_utc = datetime.utcnow() - timedelta(hours=1)

# Format the current time as a string in the required format ('%Y-%m-%dT%H:%M:%SZ')
start_time_str = current_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

# Parse the formatted string to create the start_timestamp
start_timestamp = exchange.parse8601(start_time_str)
print(start_timestamp)
# Fetch historical OHLCV data for the specified trading pair and timeframe
ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1m', since=start_timestamp, limit=None)

# Create a DataFrame from the fetched data
df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df['Timestamp'] = df['Timestamp'] + timedelta(hours=8)

# df['Volume_last_5'] = df['Volume'].rolling(window=6, min_periods=1).sum() - df['Volume']
# Print the DataFrame
print(df)


1695910222000
             Timestamp     Open     High      Low    Close     Volume
0  2023-09-28 22:11:00  1629.74  1630.25  1629.58  1630.10   1284.205
1  2023-09-28 22:12:00  1630.10  1630.11  1628.72  1629.52   2521.026
2  2023-09-28 22:13:00  1629.52  1629.52  1628.16  1628.16    853.074
3  2023-09-28 22:14:00  1628.17  1629.60  1628.17  1629.09   1406.739
4  2023-09-28 22:15:00  1629.10  1630.29  1629.09  1629.24   1888.499
5  2023-09-28 22:16:00  1629.23  1629.81  1627.68  1627.88   1939.938
6  2023-09-28 22:17:00  1627.87  1627.88  1626.44  1627.44   2954.202
7  2023-09-28 22:18:00  1627.44  1628.00  1627.04  1627.23   1298.342
8  2023-09-28 22:19:00  1627.24  1629.40  1627.24  1629.40    881.130
9  2023-09-28 22:20:00  1629.40  1634.37  1629.39  1632.69  11664.131
10 2023-09-28 22:21:00  1632.69  1632.91  1629.69  1629.69   4295.861
11 2023-09-28 22:22:00  1629.69  1630.12  1628.11  1628.46   3024.532
12 2023-09-28 22:23:00  1628.45  1629.59  1627.12  1627.28   2798.357
13 202

In [13]:
import ccxt.pro as ccxtpro
import ccxt
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
import pandas as pd

# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

async def fetch_and_update_dataframe(exchange, symbol, df):
    try:
        # Fetch the initial 1-minute bar using WebSocket
        ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')
        first_timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)

        # Calculate the new start_timestamp (1 hour earlier than the first timestamp)
        start_timestamp = first_timestamp_utc - timedelta(hours=1)

        while True:
            # Fetch the latest 1-minute bar using WebSocket
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')

            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)

            # Create a new DataFrame for the latest bar
            new_row = pd.DataFrame([[timestamp_utc, ohlcv[-1][1], ohlcv[-1][2],
                                      ohlcv[-1][3], ohlcv[-1][4], ohlcv[-1][5]]],
                                    columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

            # Append the latest bar to the existing DataFrame
            df = pd.concat([df, new_row], ignore_index=True)

            # Ensure the DataFrame contains at most the last 30 bars
            if len(df) > 30:
                df = df.iloc[-30:]

            # Print the updated DataFrame
            print(df)

            await asyncio.sleep(60)  # Sleep for 60 seconds (1 minute)
    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def main():
    symbol = 'ETH/USDT'
    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    # Create an initial DataFrame with the last 30 bars
    ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')
    first_timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)

    # Calculate the new start_timestamp (1 hour earlier than the first timestamp)
    start_timestamp = first_timestamp_utc - timedelta(hours=1)

    df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

    await fetch_and_update_dataframe(exchange, symbol, df)

# Run the async/await code in a Jupyter Notebook cell
asyncio.run(main())


                   Timestamp     Open    High      Low    Close    Volume
0        2023-09-28 14:58:00  1642.59  1643.7  1642.12  1642.29  3705.599
1  2023-09-28 14:58:00+00:00  1642.59  1643.7  1642.12  1642.44  3749.961


KeyboardInterrupt: 